In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [26]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [27]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [28]:
df_bbidding

,timestamp_utc,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,...,imbalance_price,settlement_date,settlement_period,data_provider,price_y,volume,day_ahead_price,market_price,imbalance_price_predictions,day_ahead_price_predictions
0,2021-03-04 08:00:00+00:00,50.709375,50.709375,42.008984,33.308594,15.907813,78.506250,0.194284,0.000355,5.161761,...,89.95,2021-03-04,17,APXMIDP,68.55,1201.25,73.77,68.55,57.479590,60.295418
1,2021-03-04 08:30:00+00:00,68.791812,68.791812,59.750593,50.709375,33.308594,113.314159,0.193671,0.000382,6.013144,...,89.00,2021-03-04,18,APXMIDP,69.25,1105.35,73.77,69.25,63.247158,61.874634
2,2021-03-04 09:00:00+00:00,86.874249,86.874249,77.833030,68.791812,50.709375,148.122068,0.193058,0.000454,6.864527,...,92.00,2021-03-04,19,APXMIDP,75.18,1245.05,70.86,75.18,55.339940,55.828857
3,2021-03-04 09:30:00+00:00,106.168853,106.168853,96.521551,86.874249,68.791812,186.030011,0.192385,0.000414,7.798984,...,92.00,2021-03-04,20,APXMIDP,73.19,1590.00,70.86,73.19,54.452023,55.875053
4,2021-03-04 10:00:00+00:00,125.463457,125.463457,115.816155,106.168853,86.874249,223.937954,0.191712,0.000599,8.733441,...,98.00,2021-03-04,21,APXMIDP,74.12,1415.30,63.72,74.12,59.857574,54.102753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41944,2023-08-25 23:30:00+00:00,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,...,66.01,2023-08-26,2,APXMIDP,83.91,1201.85,84.41,83.91,62.059883,75.757940
41945,2023-08-26 00:00:00+00:00,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,...,66.01,2023-08-26,3,APXMIDP,82.13,1066.95,79.96,82.13,59.772330,72.234530
41946,2023-08-26 00:30:00+00:00,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,...,66.01,2023-08-26,4,APXMIDP,78.76,997.60,79.96,78.76,59.184840,72.383970
41947,2023-08-26 01:00:00+00:00,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,...,66.01,2023-08-26,5,APXMIDP,77.47,913.25,73.84,77.47,74.162260,83.351250


In [29]:
df_bbidding.Target_MW

0        102.844285
1        162.689737
2        172.662460
3        174.278554
4        206.460381
            ...    
41944     23.648000
41945     10.865000
41946     10.505000
41947      4.486000
41948      6.580000
Name: Target_MW, Length: 41949, dtype: float64

In [30]:
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))

In [31]:
df_bbidding.Revenue_50.mean()

25645.800237410862

50% quantil gibt historisch 25437.310022730526 pro periode

In [32]:
df_bbidding["Revenue_40"] = df_bbidding["day_ahead_price"] * df_bbidding["4"]+(df_bbidding["Target_MW"]-df_bbidding["4"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["4"]))
df_bbidding["Revenue_30"] = df_bbidding["day_ahead_price"] * df_bbidding["3"]+(df_bbidding["Target_MW"]-df_bbidding["3"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["3"]))
df_bbidding["Revenue_20"] = df_bbidding["day_ahead_price"] * df_bbidding["2"]+(df_bbidding["Target_MW"]-df_bbidding["2"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["2"]))
df_bbidding["Revenue_10"] = df_bbidding["day_ahead_price"] * df_bbidding["1"]+(df_bbidding["Target_MW"]-df_bbidding["1"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["1"]))
df_bbidding["Revenue_60"] = df_bbidding["day_ahead_price"] * df_bbidding["6"]+(df_bbidding["Target_MW"]-df_bbidding["6"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["6"]))
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))
df_bbidding["Revenue_70"] = df_bbidding["day_ahead_price"] * df_bbidding["7"]+(df_bbidding["Target_MW"]-df_bbidding["7"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["7"]))
df_bbidding["Revenue_80"] = df_bbidding["day_ahead_price"] * df_bbidding["8"]+(df_bbidding["Target_MW"]-df_bbidding["8"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["8"]))
df_bbidding["Revenue_90"] = df_bbidding["day_ahead_price"] * df_bbidding["9"]+(df_bbidding["Target_MW"]-df_bbidding["9"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["9"]))

In [33]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_bbidding["Revenue_"+str(i)+"0"].mean())

Revenue_10
35247.63262120537
Revenue_20
32999.210119581716
Revenue_30
30541.56152004898
Revenue_40
27389.26853759782
Revenue_50
25645.800237410862
Revenue_60
22831.75377931324
Revenue_70
19894.275308889133
Revenue_80
15648.861394222415
Revenue_90
7704.361000305964


In [34]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [35]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



Mit optimierung 56488.5046925057 pro periode

In [36]:
df_bbidding.revenue_optimal.mean()

56847.65672443437

In [37]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

Revenue mit estimateten values (LSTM) Mit 50% quantile

In [38]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [39]:
df_bbidding.revenue_normal.mean()

25384.415205692465

LSTM 40% quantile

In [40]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['4']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [41]:
df_bbidding.revenue_normal.mean()

26910.247638908433

On newer data

In [42]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [43]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [65]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)


In [67]:
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new.timestamp = pd.to_datetime(df_api_new.timestamp)

In [70]:

# Create a continuous time series from the minimum to maximum timestamp at 30-minute intervals
full_timestamp_range = pd.date_range(start=df_api_new['timestamp'].min(), end=df_api_new['timestamp'].max(), freq='30min')
# Reindex the dataframe using the full range of timestamps
df_api_new_1 = df_api_new.set_index('timestamp').reindex(full_timestamp_range, method=None)
df_api_new_1 = df_api_new_1.reset_index().rename(columns={'index': 'timestamp'})
# Create the 'prediction_date' column based on the timestamp
df_api_new_1['prediction_date'] = df_api_new_1['timestamp'].dt.date

In [71]:
df_api_new_1

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03 22:00:00+00:00,2024-10-03,1355.0,487.0,963.0,1544.0,1330.0,867.0,326.0,1402.0,1357.0
1,2024-10-03 22:30:00+00:00,2024-10-03,788.0,584.0,1148.0,1320.0,1007.0,565.0,923.0,797.0,487.0
2,2024-10-03 23:00:00+00:00,2024-10-03,571.0,748.0,1001.0,1460.0,740.0,1017.0,1533.0,597.0,991.0
3,2024-10-03 23:30:00+00:00,2024-10-03,1476.0,1194.0,1512.0,334.0,1054.0,1572.0,744.0,1497.0,463.0
4,2024-10-04 00:00:00+00:00,2024-10-04,1352.0,1390.0,912.0,662.0,1505.0,1093.0,814.0,487.0,1340.0
...,...,...,...,...,...,...,...,...,...,...,...
763,2024-10-19 19:30:00+00:00,2024-10-19,3.0,7.0,23.0,32.0,45.0,57.0,77.0,93.0,125.0
764,2024-10-19 20:00:00+00:00,2024-10-19,0.0,11.0,25.0,36.0,42.0,60.0,82.0,103.0,132.0
765,2024-10-19 20:30:00+00:00,2024-10-19,0.0,16.0,27.0,39.0,44.0,64.0,89.0,106.0,140.0
766,2024-10-19 21:00:00+00:00,2024-10-19,0.0,11.0,29.0,42.0,51.0,75.0,97.0,113.0,151.0


In [77]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new_1,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [83]:
df_api_new_merged

,timestamp,prediction_date,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-10-03 22:00:00+00:00,2024-10-03,1355.0,487.0,963.0,1544.0,1330.0,867.0,326.0,1402.0,...,71.32,1322.45,2024-10-03 22:00:00+00:00,2024-10-03,47.0,61.0,78.47,71.32,0.866025,-0.900969
1,2024-10-03 22:30:00+00:00,2024-10-03,788.0,584.0,1148.0,1320.0,1007.0,565.0,923.0,797.0,...,71.42,1035.65,2024-10-03 22:30:00+00:00,2024-10-03,48.0,50.0,78.47,71.42,0.866025,-0.900969
2,2024-10-03 23:00:00+00:00,2024-10-03,571.0,748.0,1001.0,1460.0,740.0,1017.0,1533.0,597.0,...,66.94,1378.55,2024-10-03 23:00:00+00:00,2024-10-04,1.0,93.5,71.36,66.94,0.965926,-0.900969
3,2024-10-03 23:30:00+00:00,2024-10-03,1476.0,1194.0,1512.0,334.0,1054.0,1572.0,744.0,1497.0,...,60.03,1539.90,2024-10-03 23:30:00+00:00,2024-10-04,2.0,93.5,71.36,60.03,0.965926,-0.900969
4,2024-10-04 00:00:00+00:00,2024-10-04,1352.0,1390.0,912.0,662.0,1505.0,1093.0,814.0,487.0,...,65.21,1818.40,2024-10-04 00:00:00+00:00,2024-10-04,3.0,93.0,72.80,65.21,1.000000,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,2024-10-19 19:30:00+00:00,2024-10-19,3.0,7.0,23.0,32.0,45.0,57.0,77.0,93.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,-0.222521
773,2024-10-19 20:00:00+00:00,2024-10-19,0.0,11.0,25.0,36.0,42.0,60.0,82.0,103.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
774,2024-10-19 20:30:00+00:00,2024-10-19,0.0,16.0,27.0,39.0,44.0,64.0,89.0,106.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
775,2024-10-19 21:00:00+00:00,2024-10-19,0.0,11.0,29.0,42.0,51.0,75.0,97.0,113.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,-0.222521


In [78]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag48h"] = df_api_new_merged1["market_price"].shift(96)
df_api_new_merged1.loc[:,"imbalance_price_lag48h"] = df_api_new_merged1["imbalance_price"].shift(96)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag48h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [86]:
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag1week,volume_lag48h
0,2024-10-10 17:30:00+00:00,121.68,108.01,2667.85,38.0,-2.588190e-01,-0.900969,19.0,18.0,15.0,...,10.0,9.0,6.0,4.0,2.0,124.890000,55.09,55.09,78.47,2360.40
1,2024-10-10 18:00:00+00:00,113.01,91.93,1878.60,39.0,-1.836970e-16,-0.900969,19.0,17.0,14.0,...,10.0,8.0,6.0,3.0,1.0,124.890000,56.63,0.00,78.47,1593.60
2,2024-10-10 18:30:00+00:00,102.06,91.93,1826.20,40.0,-1.836970e-16,-0.900969,19.0,17.0,14.0,...,10.0,8.0,6.0,3.0,1.0,119.000153,49.02,93.90,71.36,1503.15
3,2024-10-10 19:00:00+00:00,95.80,82.15,1598.15,41.0,2.588190e-01,-0.900969,19.0,17.0,14.0,...,10.0,8.0,5.0,3.0,1.0,124.890000,46.59,5.00,71.36,1608.85
4,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,2.588190e-01,-0.900969,19.0,17.0,14.0,...,10.0,8.0,5.0,3.0,1.0,103.500000,37.22,0.00,72.80,1415.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,2.588190e-01,0.623490,850.0,921.0,980.0,...,1060.0,1081.0,1104.0,1123.0,1142.0,105.000000,0.00,159.00,58.26,0.00
244,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,5.000000e-01,0.623490,897.0,964.0,1010.0,...,1074.0,1097.0,1111.0,1129.0,1151.0,104.900000,0.00,70.00,53.59,0.00
245,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,5.000000e-01,0.623490,910.0,984.0,1040.0,...,1101.0,1120.0,1122.0,1139.0,1154.0,64.000000,0.00,70.00,53.59,0.00
246,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,7.071068e-01,0.623490,938.0,1034.0,1069.0,...,1124.0,1135.0,1139.0,1152.0,1164.0,33.770000,0.00,129.00,53.59,0.00


In [79]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [87]:
wind_total

,timestamp_utc,settlement_date,settlement_period,boa,generation_mw
0,2020-12-31 23:00:00+00:00,2020-12-31,47,0.0,0.000
1,2020-12-31 23:30:00+00:00,2020-12-31,48,0.0,0.000
2,2021-01-01 00:00:00+00:00,2021-01-01,1,0.0,92.879
3,2021-01-01 00:30:00+00:00,2021-01-01,2,0.0,75.641
4,2021-01-01 01:00:00+00:00,2021-01-01,3,0.0,58.573
...,...,...,...,...,...
65947,2024-10-10 19:30:00+00:00,2024-10-10,42,0.0,361.670
65948,2024-10-10 20:00:00+00:00,2024-10-10,43,0.0,410.810
65949,2024-10-10 20:30:00+00:00,2024-10-10,44,0.0,423.910
65950,2024-10-10 21:00:00+00:00,2024-10-10,45,0.0,417.860


In [88]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,imbalance_price_lag48h,day_ahead_price_lag1week,volume_lag48h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind
0,2024-10-10 17:30:00+00:00,121.68,108.01,2667.85,38.0,-2.588190e-01,-0.900969,19.0,18.0,15.0,...,55.09,78.47,2360.40,0.0,2956.745251,2779.347700,2024-10-10,38,0.0,404.340
1,2024-10-10 18:00:00+00:00,113.01,91.93,1878.60,39.0,-1.836970e-16,-0.900969,19.0,17.0,14.0,...,0.00,78.47,1593.60,0.0,2956.745251,2779.347687,2024-10-10,39,0.0,482.370
2,2024-10-10 18:30:00+00:00,102.06,91.93,1826.20,40.0,-1.836970e-16,-0.900969,19.0,17.0,14.0,...,93.90,71.36,1503.15,0.0,2956.745251,2779.347676,2024-10-10,40,0.0,413.779
3,2024-10-10 19:00:00+00:00,95.80,82.15,1598.15,41.0,2.588190e-01,-0.900969,19.0,17.0,14.0,...,5.00,71.36,1608.85,0.0,2956.745251,2779.347666,2024-10-10,41,0.0,270.840
4,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,2.588190e-01,-0.900969,19.0,17.0,14.0,...,0.00,72.80,1415.55,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670
5,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,5.000000e-01,-0.900969,19.0,16.0,14.0,...,29.53,72.80,1765.45,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810
6,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,5.000000e-01,-0.900969,19.0,16.0,14.0,...,34.95,65.30,1913.65,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910
7,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,7.071068e-01,-0.900969,19.0,17.0,14.0,...,0.00,65.30,1913.65,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860
8,2024-10-10 21:30:00+00:00,78.34,91.45,1806.70,46.0,7.071068e-01,-0.900969,19.0,17.0,14.0,...,42.29,71.46,2309.70,0.0,2956.745251,2779.347587,2024-10-10,46,0.0,348.410


In [89]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag1week","volume_lag48h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

In [90]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_26056\1576695596.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_p

In [91]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_26056\823899303.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_pr

In [92]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [93]:
df_api_new_merged2.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag48h',
       'imbalance_price_lag48h', 'day_ahead_price_lag1week', 'volume_lag48h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW'],
      dtype='object')

In [94]:
df_api_new_merged2["Revenue_40"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["4"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"]))
df_api_new_merged2["Revenue_30"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["3"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"]))
df_api_new_merged2["Revenue_20"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["2"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"]))
df_api_new_merged2["Revenue_10"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["1"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"]))
df_api_new_merged2["Revenue_60"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["6"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"]))
df_api_new_merged2["Revenue_50"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["5"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"]))
df_api_new_merged2["Revenue_70"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["7"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"]))
df_api_new_merged2["Revenue_80"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["8"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"]))
df_api_new_merged2["Revenue_90"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["9"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"]))

In [95]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_api_new_merged2["Revenue_"+str(i)+"0"].mean())

Revenue_10
24224.818281771375
Revenue_20
24100.63385803673
Revenue_30
23962.868955768095
Revenue_40
23841.695976477895
Revenue_50
23783.732041632134
Revenue_60
23638.97675123083
Revenue_70
23500.399371940628
Revenue_80
23365.216580783104
Revenue_90
23254.732512604016


In [96]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [97]:
df_bbidding.revenue_normal.mean()

26910.247638908433

monte carlo: 

In [ ]:
import numpy as np

# Define revenue calculation function
def calculate_revenue(trade, actual, DAP, SSP):
    return trade * DAP + (actual - trade) * (SSP - 0.07 * (actual - trade))

# Monte Carlo simulation to estimate expected revenue
def monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP, num_samples=1000):
    expected_revenue = []
    
    # Sample from the probabilistic distribution (Monte Carlo simulation)
    sampled_actuals = actual_distribution(num_samples)
    
    for trade in trade_values:
        # Calculate revenue for each sample and trade value
        revenues = [calculate_revenue(trade, actual, DAP, SSP) for actual in sampled_actuals]
        # Store the average expected revenue for this trade value
        expected_revenue.append(np.mean(revenues))
    
    return trade_values, expected_revenue

# Example usage
# Define input values (these would be dynamic in practice)
DAP = 50  # Day-Ahead Price
SSP = 45  # System Sell Price

# Generate probabilistic forecast (e.g., Normal distribution with mean and std dev from your forecast)
actual_mean = 100  # Assume mean production is 100 MWh
actual_std = 10    # Assume a standard deviation of 10 MWh
actual_distribution = lambda n: np.random.normal(actual_mean, actual_std, n)

# Define the range of Trade values to evaluate
trade_values = np.linspace(80, 120, 50)  # Evaluate Trade between 80 and 120 MWh

# Run Monte Carlo simulation
trade_values, expected_revenues = monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP)

# Find the optimal trade value that maximizes expected revenue
optimal_trade = trade_values[np.argmax(expected_revenues)]

print(f"Optimal Trade Value: {optimal_trade} MWh")


1. Dynamic Programming Approach


In [ ]:
for t in range(T-1, -1, -1):  # Start at last time step, move backwards
    for s in state_space:  # For each state in the state space
        max_value = -inf
        for a in action_space:  # For each possible trade action
            next_state = transition_function(s, a)
            immediate_reward = revenue(s, a)
            future_value = gamma * V[next_state]  # Discounted future value
            total_value = immediate_reward + future_value
            if total_value > max_value:
                max_value = total_value
                best_action[s] = a  # Store the best action for state s
        V[s] = max_value
